<a href="https://colab.research.google.com/github/ysikalie/colorization/blob/master/full_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import tensorflow as tf
import keras
from keras.applications.densenet import DenseNet201
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate,Dense
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

# 新段落

In [0]:
!mkdir train

In [0]:
# Get images
X = []
for filename in os.listdir('Train/'):
    X.append(img_to_array(load_img('Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X




In [0]:
# embed_input = Input(shape=(1000,))

#Mid-Level features network
gray_encoder_input = Input(shape=(256, 256, 1,))
gray_encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_input)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)

#Add
#Global Features network
color_encoder_input = Input(shape=(256, 256, 3,))
color_encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(color_encoder_input)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
color_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(color_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
color_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(color_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
color_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(color_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
color_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(color_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
color_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(color_encoder_output)
gray_encoder_output = BatchNormalization()(gray_encoder_output)
color_encoder_output = Flatten(name='flatten')(color_encoder_output)
color_encoder_output = Dense(1024, activation='sigmoid')(color_encoder_output)
color_encoder_output = Dense(512, activation='sigmoid')(color_encoder_output)
color_encoder_output = Dense(256, activation='sigmoid')(color_encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(color_encoder_output) 
fusion_output = Reshape(([32, 32, 256]))(fusion_output)
fusion_output = concatenate([gray_encoder_output, fusion_output], axis=3) 
 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[gray_encoder_input, color_encoder_input], outputs=decoder_output)
model.summary()


In [0]:


# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 1

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
#         embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        print([X_batch,lab_batch], Y_batch)
        yield ([X_batch, lab_batch], Y_batch)


from keras.optimizers import RMSprop
rmsprop = RMSprop(lr=1e-3)
model.compile(optimizer=rmsprop, loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=1 ,steps_per_epoch=1)

Epoch 1/1


TypeError: ignored

In [0]:
！ nvidia-smi